## Carga Data TERMINO X MATERIA

In [2]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [3]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
#creacion dataframe con los datos de Términos por Materia Penal de los años 2015 a 2020

dataframes = []

for archivo in archivos:
    if archivo.find("Términos por Materia Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', low_memory = True)
        dataframes.append(df) 
        
df_termino_materia = pd.concat(dataframes, axis = 0)

In [5]:
df_termino_materia.columns

Index(['N°', 'CÓD. CORTE', 'CORTE', 'CÓD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'CÓD. MATERIA', 'MATERIA', 'FECHA INGRESO',
       'FECHA TÉRMINO', 'MES TÉRMINO', 'AÑO TÉRMINO', 'MOTIVO DE TÉRMINO',
       'DURACIÓN CAUSA', 'TOTAL TÉRMINOS', 'SISTEMA'],
      dtype='object')

In [6]:
# Estandarización de nombres de variables

df_termino_materia.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                     'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                     'CÓD. MATERIA':'COD. MATERIA',
                                     'MOTIVO DE TÉRMINO':'MOTIVO TERMINO',
                                     'DURACIÓN CAUSA':'DURACION CAUSA',
                                     'FECHA TÉRMINO':'FECHA TERMINO'
                                    },
                          inplace = True)
df_termino_materia.drop(['N°','MES TÉRMINO','AÑO TÉRMINO','SISTEMA','TOTAL TÉRMINOS'],axis = 'columns', inplace = True)

In [7]:
# TRANSFORMAMOS DE FLOAT A INTEGER

df_termino_materia['COD. CORTE'] = df_termino_materia['COD. CORTE'].fillna(0).astype(np.int16)
df_termino_materia['COD. TRIBUNAL'] = df_termino_materia['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_termino_materia['COD. MATERIA'] = df_termino_materia['COD. MATERIA'].fillna(0).astype(np.int16)
df_termino_materia['DURACION CAUSA'] = df_termino_materia['DURACION CAUSA'].fillna(0).astype(np.int16)

In [8]:
# Transformamos formato fecha

df_termino_materia['FECHA INGRESO'] = df_termino_materia['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)
df_termino_materia['FECHA TERMINO'] = df_termino_materia['FECHA TERMINO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 3606632/3606632 [00:06<00:00, 578548.41it/s]


In [9]:
# Elimino espacios en las columnas tipo objetos

df_termino_materia = df_termino_materia.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 12/12 [00:29<00:00,  2.43s/it]


In [10]:
# Elimino tildes de object

cols = df_termino_materia.select_dtypes(include = ["object"]).columns
df_termino_materia[cols] = df_termino_materia[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 6/6 [00:53<00:00,  8.87s/it]


In [11]:
# Categorizar variables

df_termino_materia['CORTE'] = df_termino_materia['CORTE'].astype('category')
df_termino_materia['TIPO CAUSA'] = df_termino_materia['TIPO CAUSA'].astype('category')
df_termino_materia['MOTIVO TERMINO'] = df_termino_materia['MOTIVO TERMINO'].astype('category')

In [12]:
# Reset el index para realizar feather

df_termino_materia.reset_index(inplace = True)

In [13]:
df_termino_materia

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,FECHA TERMINO,MOTIVO TERMINO,DURACION CAUSA
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,SENTENCIA,60
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,SENTENCIA,60
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,SENTENCIA,149
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,SENTENCIA,63
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,SENTENCIA,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3606627,721902,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2015,Ordinaria,901,"CUASIDELITO DE LESIONES: ART 490, 491 INC 2° Y...",2015-12-17,2016-05-21,SENTENCIA,156
3606628,721903,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2015,Ordinaria,12077,NO DAR CUENTA DE ACCIDENTE DE TRANSITOART. 195...,2015-12-17,2016-05-21,SENTENCIA,156
3606629,721904,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,98-2016,Ordinaria,10008,PORTE DE ARMA PROHIBIDA (ART. 14 INC. 1°),2016-07-27,2016-11-08,SENTENCIA,104
3606630,721905,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,99-2015,Ordinaria,12082,AMENAZASA CARABINEROS (ART. 417 COD. J.MILITAR).,2015-12-17,2016-01-18,SENTENCIA,32


In [15]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_termino_materia.to_feather(f'{path_interim}/TerminoMateria_feather')